# IS434 Twitter/X Bot Detection Project

This Project aims to use machine learning and analytics techniques to detect bot accounts on the Twitter/X platform 

# 0. Install Libraries

In [33]:
!pip install pgmpy
!pip install gensim
!pip install nltk
!pip install imblearn
!pip install keras
!pip install scikit-learn
!pip install tensorflow

  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached werkzeug-3.0.4-py3-none-any.whl.metadata (3.7 kB)
   ---------------------------------------- 0.0/385.2 MB ? eta -:--:--
   ---------------------------------------- 1.0/385.2 MB 6.3 MB/s eta 0:01:02
   ---------------------------------------- 2.4/385.2 MB 5.6 MB/s eta 0:01:09
   ---------------------------------------- 2.9/385.2 MB 6.0 MB/s eta 0:01:04
   ---------------------------------------- 3.9/385.2 MB 4.7 MB/s eta 0:01:22
    --------------------------------------- 5.5/385.2 MB 5.3 MB/s eta 0:01:12
    --------------------------------------- 7.3/385.2 MB 5.9 MB/s eta 0:01:05
    ----------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.66.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.5 which is incompatible.


In [11]:
import pandas as pd
df_train = pd.read_csv('data/labeled_data.csv')
# drop unnamed
df_train = df_train.drop(columns=['Unnamed: 0'])
# show 10 entires from row 47 onwards, and make the tweet column wider, align left
pd.set_option('display.max_colwidth', 150)
df_train[67:83].style.set_properties(**{'text-align': 'left'})

,count,hate_speech,offensive_language,neither,class,tweet
67,3,0,1,2,2,"""@Allyhaaaaa: Lemmie eat a Oreo & do these dishes."" One oreo? Lol"
68,3,0,3,0,1,"""@Almightywayne__: @JetsAndASwisher @Gook____ bitch fuck u http://t.co/pXmGA68NC1"" maybe you'll get better. Just http://t.co/TPreVwfq0S"
69,3,1,2,0,1,"""@Almightywayne__: Fuck Red Malone man bitch ass niggah"" could you please use complete sentences?"
70,3,0,0,3,2,"""@ArizonasFinest6: Why the eggplant emoji doe?""y he say she looked like scream lmao"
71,3,0,3,0,1,"""@AutoWorId: Hennessey Venom GT 🙈 http://t.co/i8eGMnKaJ9"" that's one sexy bitch"
72,3,0,3,0,1,"""@BOSSBYTCHH: Him seh me pussy wetter then a shower curtain....#ahmesehwetness""<lmao!!"
73,3,0,3,0,1,"""@BRO_HEN314: #Eaglesnation and every #Eagles need to see that pic I just posted because that bitch just said the most racist shit"""
74,3,1,2,0,1,"""@BVSEDCHINK: Yo fuck skateboarding, all y'all some wood pushing faggots man, ball is life http://t.co/VBCzP6HMT7"" @AmJemieBenn"
75,3,0,1,2,2,"""@BabyAnimalPics: baby monkey bathtime http://t.co/7KPWAdLF0R"" Awwwwe! This is soooo ADORABLE!"
76,3,0,3,0,1,"""@BaylaaGottaBody: 😂😂😂😂 I ain't shit ."" Damn Skippy lol"


## 1. Data Processing

In [12]:
import pandas as pd
import re
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import numpy as np

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

# Load the dataset
df_train = pd.read_csv('data/main.csv')
df_finetune = pd.read_csv('data/finetune.csv')

# Function to clean the tweets (basic preprocessing)
def preprocess_text(text):
    text = str(text)  
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^A-Za-z0-9 ]+', '', text)  # Remove special characters
    text = text.lower()  # Lowercase the text
    tokens = word_tokenize(text)  # Tokenize the text
    stop_words = set(stopwords.words('english'))  # Define stopwords
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return tokens

# Preprocess each tweet in the dataset
df_train['cleaned_tweet'] = df_train['tweet'].apply(preprocess_text)
df_finetune['cleaned_tweet'] = df_finetune['tweet'].apply(preprocess_text)

# concatenate the two datasets
df_combined = pd.concat([df_train, df_finetune])

# Train the Word2Vec model
model = Word2Vec(sentences=df_combined['cleaned_tweet'], vector_size=100, window=5, min_count=2, workers=4)

# Save the model for future use
model.save("word2vec_tweet_model.model")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\benhz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\benhz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\benhz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences

# Set your maximum sequence length (e.g., 30 words per tweet)
max_seq_length = 30
embedding_dim = 100  # Same as the vector_size of Word2Vec

# get word embeddings for tweet
def get_tweet_embeddings(tweet, word2vec_model, embedding_dim):
    embeddings = []
    for word in tweet:
        if word in word2vec_model.wv:
            embeddings.append(word2vec_model.wv[word])  # Get the word vector for each word
        else:
            embeddings.append(np.zeros(embedding_dim))  # Handle out-of-vocabulary words
    return embeddings

# Convert the entire dataset into sequences of word embeddings
tweet_embeddings = [get_tweet_embeddings(tweet, model, embedding_dim) for tweet in df_train['cleaned_tweet']]

# Pad the sequences to ensure they are all of the same length (max_seq_length)
padded_tweet_embeddings = pad_sequences(tweet_embeddings, maxlen=max_seq_length, dtype='float32', padding='post', truncating='post')


In [35]:
def tweet_to_single_vector(tweet, model):
    word_vectors = [model.wv[word] for word in tweet if word in model.wv]
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0) 
    else:
        return np.zeros(model.vector_size)  



In [4]:
df_train['tweet_vectors_single'] = df_train['cleaned_tweet'].apply(lambda x: tweet_to_single_vector(x, model))
# df_test['tweet_vectors'] = df_test['cleaned_tweet'].apply(lambda x: tweet_to_vector(x, model))

# Convert the column of vectors into a numpy array (for model training)
tweet_vectors = np.array(df_train['tweet_vectors'].to_list())
# tweet_vectors_test = np.array(df_test['tweet_vectors'].to_list())

In [30]:
df_train

,id,label,tweet,cleaned_tweet,tweet_vectors
0,1,0,@user when a father is dysfunctional and is s...,"[user, father, dysfunctional, selfish, drags, ...","[-0.2000133, 0.4018536, 0.02162266, 0.276889, ..."
1,2,0,@user @user thanks for #lyft credit i can't us...,"[user, user, thanks, lyft, credit, cant, use, ...","[-0.07638466, 0.23242302, 0.05301269, 0.322066..."
2,3,0,bihday your majesty,"[bihday, majesty]","[-0.3395153, 0.44796824, 0.26759112, 0.1310546..."
3,4,0,#model i love u take with u all the time in ...,"[model, love, u, take, u, time, ur]","[-0.5439526, 1.0469629, -0.45181212, 1.1627967..."
4,5,0,factsguide: society now #motivation,"[factsguide, society, motivation]","[-0.2341751, 0.30308622, -0.067414165, -0.1112..."
...,...,...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...,"[ate, user, isz, youuu]","[-0.13588251, 0.23949353, 0.12024341, 0.275756..."
31958,31959,0,to see nina turner on the airwaves trying to...,"[see, nina, turner, airwaves, trying, wrap, ma...","[-0.10440235, 0.16917214, 0.0024858634, 0.1836..."
31959,31960,0,listening to sad songs on a monday morning otw...,"[listening, sad, songs, monday, morning, otw, ...","[-0.21998382, 0.52339184, 0.11752925, 0.245401..."
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,...","[user, sikh, temple, vandalised, calgary, wso,...","[-0.23926866, 0.3213547, -0.081885934, -0.0648..."


Check for class imbalance

In [8]:
import pandas as pd

class_counts = df_train['label'].value_counts()
print(class_counts)


label
0    29720
1     2242
Name: count, dtype: int64


In [39]:
# split train and test data
from sklearn.model_selection import train_test_split

X = tweet_vectors
X_rnn = padded_tweet_embeddings
y = df_train['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Over-sampling train set

In [40]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
print(f'Shape of X_train_resampled: {X_train_resampled.shape}')

Shape of X_train_resampled: (47566, 100)


Split test and val set

In [41]:
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [26]:
print(f'Shape of X_test: {X_test.shape}')
print(f'Shape of X_val: {X_val.shape}')

Shape of X_test: (3197, 100)
Shape of X_val: (3196, 100)


# Modelling 

## RNN

In [43]:
X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(X_rnn, y, test_size=0.2, random_state=42)
X_val_rnn, X_test_rnn, y_val_rnn, y_test_rnn = train_test_split(X_test_rnn, y_test_rnn, test_size=0.5, random_state=42)

print(f'Shape of X_train_rnn: {X_train_rnn.shape}')
print(f'Shape of X_test_rnn: {X_test_rnn.shape}')
print(f'Shape of X_val_rnn: {X_val_rnn.shape}')

Shape of X_train_rnn: (25569, 30, 100)
Shape of X_test_rnn: (3197, 30, 100)
Shape of X_val_rnn: (3196, 30, 100)


In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR


Hyper Params

In [45]:
class Args:
  epochs = 20
  lr = 0.001
  use_cuda=False
  gamma = 0.7
  log_interval = 10
  seed = 1

args = Args()

device = torch.device("cuda" if args.use_cuda else "cpu")

print(f'Using device: {device}')

Using device: cpu


Model Training

In [50]:
# load data onto torch
X_train_rnn = torch.tensor(X_train_rnn).float()
y_train_rnn = torch.tensor(y_train_rnn.values).long()
X_val_rnn = torch.tensor(X_val_rnn).float()
y_val_rnn = torch.tensor(y_val_rnn.values).long()
X_test_rnn = torch.tensor(X_test_rnn).float()
y_test_rnn = torch.tensor(y_test_rnn.values).long()

print(f'Shape of X_train_rnn: {X_train_rnn.shape}')

Shape of X_train_rnn: torch.Size([25569, 30, 100])


In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class RNNModel(nn.Module):
    def __init__(self):
        super(RNNModel, self).__init__()
        
        self.lstm1 = nn.LSTM(input_size=100, hidden_size=128, num_layers=2, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(0.3)
        self.fc1 = nn.Linear(128 * 2, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 3)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.dropout(x)
        x = x[:, -1, :]
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x


In [52]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            

In [53]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    return 100. * correct / len(test_loader.dataset)

In [54]:
# Create the model
model = RNNModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=args.lr)
scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)

# Create the data loaders
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train_rnn, y_train_rnn), batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_val_rnn, y_val_rnn), batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_test_rnn, y_test_rnn), batch_size=64, shuffle=False)

# Training loop
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    acc = test(args, model, device, val_loader)
    scheduler.step()

# Test the model
test(model, device, test_loader)

Train Epoch: 1 [0/25569 (0%)]	Loss: 1.119942
Train Epoch: 1 [640/25569 (2%)]	Loss: 0.310309
Train Epoch: 1 [1280/25569 (5%)]	Loss: 0.330944
Train Epoch: 1 [1920/25569 (8%)]	Loss: 0.103862
Train Epoch: 1 [2560/25569 (10%)]	Loss: 0.453529
Train Epoch: 1 [3200/25569 (12%)]	Loss: 0.267973
Train Epoch: 1 [3840/25569 (15%)]	Loss: 0.279113
Train Epoch: 1 [4480/25569 (18%)]	Loss: 0.275416
Train Epoch: 1 [5120/25569 (20%)]	Loss: 0.267811
Train Epoch: 1 [5760/25569 (22%)]	Loss: 0.112255
Train Epoch: 1 [6400/25569 (25%)]	Loss: 0.149453
Train Epoch: 1 [7040/25569 (28%)]	Loss: 0.353479
Train Epoch: 1 [7680/25569 (30%)]	Loss: 0.282026
Train Epoch: 1 [8320/25569 (32%)]	Loss: 0.571090
Train Epoch: 1 [8960/25569 (35%)]	Loss: 0.173916
Train Epoch: 1 [9600/25569 (38%)]	Loss: 0.237981
Train Epoch: 1 [10240/25569 (40%)]	Loss: 0.200826
Train Epoch: 1 [10880/25569 (42%)]	Loss: 0.162034
Train Epoch: 1 [11520/25569 (45%)]	Loss: 0.195918
Train Epoch: 1 [12160/25569 (48%)]	Loss: 0.155664
Train Epoch: 1 [12800/25

92.89959336878323

## RF Classifier

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.feature_selection import RFE
from joblib import dump, load

In [57]:
rf_classifier = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Setup grid search
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
y
grid_search.fit(X_train_resampled, y_train_resampled)

# Best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

best_rf = grid_search.best_estimator_


# the grid search aims to 

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best cross-validation score: 0.99


Save & load model

In [61]:
dump(best_rf, 'model/rf_model.joblib')


Model Training

In [3]:
best_rf = load('model/rf_model.joblib')

In [5]:
best_rf.fit(X_train_resampled, y_train_resampled)

NameError: name 'X_train_resampled' is not defined

In [65]:
y_pred = best_rf.predict(X_test)
RF_accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", RF_accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.9396309039724742
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      2970
           1       0.70      0.26      0.38       227

    accuracy                           0.94      3197
   macro avg       0.82      0.63      0.68      3197
weighted avg       0.93      0.94      0.93      3197



In [18]:
# make prediction on text
import pandas as pd
import re
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import numpy as np

def preprocess_text(text):
    text = str(text)  
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^A-Za-z0-9 ]+', '', text)  # Remove special characters
    text = text.lower()  # Lowercase the text
    tokens = word_tokenize(text)  # Tokenize the text
    stop_words = set(stopwords.words('english'))  # Define stopwords
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return tokens

def tweet_to_single_vector(tweet, model):
    word_vectors = [model.wv[word] for word in tweet if word in model.wv]
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0) 
    else:
        return np.zeros(model.vector_size)  

def predict_rf(text):
    cleaned_text = preprocess_text(text)
    vector = tweet_to_single_vector(cleaned_text, model)
    return best_rf.predict([vector])[0]

predict_rf("I am happy")


0

## Bayes

In [19]:
# Import relevant packages
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
import sys

c:\Users\benhz\Documents\GitHub\IS434\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
# We first create a model which containts edges of the graph
model = BayesianModel([('Username', 'Bot Label'), ('tweet_vectors', 'Bot Label'), ('Retweet Count', 'Bot Label'), ('Mention Count', 'Bot Label'), ('Follower Count', 'Bot Label'), ('Verified', 'Bot Label')], ('Location', 'Bot Label'), ('Created At', 'Bot Label'),('Hashtags', 'Bot Label'))


TypeError: BayesianModel.__init__() takes from 1 to 3 positional arguments but 5 were given